<a href="https://colab.research.google.com/github/tyukei/AI/blob/master/report2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#第2回 レポート課題 (12/20 23:55 締切)


# 課題1
### トピック9のnotebook1で示したRNNを用いたIMDBデータのP/N判定のコードを参考に、複数のネットワーク構造を設定して性能の違いを調べよ。また、調べたネットワーク構造の中から比較的性能が高くて学習時間が短いものをひとつ選び、学習データが極端に少ない状況からすべての学習データを用いる状況まで、段階的にデータを増やして学習データの量と識別器の性能の関係を調べよ。 

RNNとはRecursive Neural Network(再帰型ニューラルネットワーク)の略である。

Recursiveとあるように中間層で演算結果を再帰的に自身に出力する

具体的には、一時刻前の中間層の出力を次の入力と結合して中間層に入力する。

そのため、前の単語が次の単語に影響をあたえる自然言語処理に適していると言われる。


ディープニューラルネットワークでの識別を Keras でコーディングします。Google ColabでGPUを使用するときは、「ランタイム」->「ランタイムのタイプを変更」-> ハードウェアアクセラレータ -> GPU を選びます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime, os
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import np_utils
from tensorflow.keras import layers

In [ ]:
%load_ext tensorboard

In [ ]:
max_features = 10000
maxlen = 50
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

17464789/17464789 [==============================] - 2s 0us/step


In [ ]:
word_index = keras.datasets.imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_train[0]])
decoded_review

1641221/1641221 [==============================] - 0s 0us/step


"grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all"

In [ ]:
model1 = keras.Sequential([
    layers.Embedding(max_features, 128),
    layers.SimpleRNN(64),
    layers.Dense(1, activation='sigmoid')
]) 
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                12352     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1,292,417
Trainable params: 1,292,417
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model1.fit(X_train, y_train, epochs=2, batch_size=32,validation_split=0.2)

Epoch 1/2
625/625 [==============================] - 26s 37ms/step - loss: 0.5303 - acc: 0.7243 - val_loss: 0.4435 - val_acc: 0.7958
Epoch 2/2
625/625 [==============================] - 23s 37ms/step - loss: 0.3189 - acc: 0.8684 - val_loss: 0.4951 - val_acc: 0.7918


In [ ]:
test_loss, test_acc = model1.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

782/782 [==============================] - 5s 6ms/step - loss: 0.4842 - acc: 0.7967
Test accuracy: 0.7967


In [ ]:
model2 = keras.Sequential([
    layers.Embedding(max_features, 128),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model2.fit(X_train, y_train, epochs=2, batch_size=32,validation_split=0.2)

Epoch 1/2
625/625 [==============================] - 11s 10ms/step - loss: 0.4627 - acc: 0.7744 - val_loss: 0.4043 - val_acc: 0.8124
Epoch 2/2
625/625 [==============================] - 6s 9ms/step - loss: 0.3025 - acc: 0.8728 - val_loss: 0.4374 - val_acc: 0.8086


In [ ]:
test_loss, test_acc = model2.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

782/782 [==============================] - 4s 5ms/step - loss: 0.4353 - acc: 0.8110
Test accuracy: 0.811


# 課題2
### トピック9のnotebook2のコードを参考に、事前学習モデルを用いる設定で、学習データが極端に少ない状況からすべての学習データを用いる状況まで、段階的にデータを増やして学習データの量と識別器の性能の関係を調べよ。